In [1]:
import os 
os.chdir("../")

In [2]:
import torch
from dataclasses import dataclass
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from src.vision_Transformer.utils.common import read_yaml , create_directories
from src.vision_Transformer.constants import *
from src.vision_Transformer.logging import logger
import tqdm 
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir : Path
    test_accuracy : Path
    data_dir : Path

    batch_size : int
    epochs : int 
    learning_rate : float
    patch_size : int
    num_classes : int
    image_size : int 
    channels : int
    embed_dim : int
    num_heads: int
    depth : int
    mlp_dim : int
    dropout_rate : float
    weight_decay : float

In [4]:
class ConfigurationManager:
    def __init__(self , config_file_path = CONFIG_FILE_PATH , params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evalutaion_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir , 
            test_accuracy= config.test_accuracy,
            data_dir= config.data_dir,

            batch_size= params.BATCH_SIZE,
            epochs = params.EPOCHS,
            learning_rate = params.LEARNING_RATE,
            patch_size = params.PATCH_SIZE,
            num_classes = params.NUM_CLASSES,
            image_size = params.IMAGE_SIZE,
            channels = params.CHANNELS,
            embed_dim  = params.EMBED_DIM,
            num_heads = params.NUM_HEADS,
            depth = params.DEPTH,
            mlp_dim = params.MLP_DIM,
            dropout_rate = params.DROPOUT_RATE,
            weight_decay  = params.WEIGHT_DECAY
        )

        return model_evaluation_config

In [5]:
from src.vision_Transformer.Components.ViT_Component.Vision_Transformer_Class import Vision_Transformer_Class
from tqdm.auto import tqdm
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

d:\anaconda3\envs\pytorch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from src.vision_Transformer.logging import logger

In [7]:
class DataTransformation:
    def __init__(self , config : ModelEvaluationConfig):
        self.config = config

    def data_augmentation(self):
        self.after_transforms = transforms.Compose([
            transforms.RandomCrop(32 , padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2 ,contrast= 0.2, saturation=0.2 , hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5]*3 , std = [0.5]*3)
        ])
    
    def transformed_dataset(self):
        transformed_test_dataset = datasets.CIFAR10(
            root = self.config.data_dir,
            train = False,
            download= False,
            transform= self.after_transforms,
        )
        logger.info(f"Test Dataset Transformed Successfully")
        print(f"Test Dataset Transformed Successfully")

        return transformed_test_dataset

In [8]:
config = ConfigurationManager()
model_eval_config = config.get_model_evalutaion_config()

data_transfromation = DataTransformation(model_eval_config)
data_transfromation.data_augmentation()

test_dataset = data_transfromation.transformed_dataset()

[2025-08-09 14:24:20,219 : INFO : common  : yaml file config\config.yaml was read succesfully]
[2025-08-09 14:24:20,226 : INFO : common  : yaml file params.yaml was read succesfully]
[2025-08-09 14:24:20,227 : INFO : common  : Created directory at : artifacts]
[2025-08-09 14:24:20,227 : INFO : common  : Created directory at : artifacts/model_evaluation]
[2025-08-09 14:24:20,681 : INFO : 1566778438  : Test Dataset Transformed Successfully]
Test Dataset Transformed Successfully


In [9]:
import json

In [12]:
class ModelEvaluation:
    def __init__(self, config : ConfigurationManager , test_dataset):
        self.config = config
        
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        self.test_dataset = test_dataset

        self.test_loader =  DataLoader(test_dataset , batch_size = self.config.batch_size ,shuffle = False ,pin_memory= True)

        self.model = torch.load("").to(self.device)


    # ------------------------------------------------------
        self.criterion = nn.CrossEntropyLoss(label_smoothing= 0.1)
        self.optimizer = optim.AdamW(self.model.parameters() , lr = float(self.config.learning_rate) , weight_decay= self.config.weight_decay)
        self.scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer= self.optimizer , T_max= self.config.epochs)
    
    # ---------------------------------------------------------------------------
    def show_model(self):
        print("\n\n------------------------------->Model Configuration<------------------------------------")
        print("\n", self.model)

        print("\n\n\n", self.device)
        
    def evaluate(self):
        #set the model to the testing mode 
        self.model.eval()
        

        total_loss , correct_prediction = 0 , 0

        with torch.no_grad():
            for x , y in self.test_loader:
                x , y = x.to(self.device) , y.to(self.device)

                output = self.model(x)

                loss = self.criterion(output , y)

                correct_prediction += (output.argmax(1) == y).sum().item()

        return correct_prediction / len(self.test_loader.dataset)
    

    def model_test_pipeline(self):
        test_accuracy_log= []
        test_accuracy_file = self.config.test_accuray

        os.makedirs(os.path.dirname(test_accuracy_file) , exist_ok= True)


        for epoch in tqdm(range(self.config.epochs)):

            test_acc = self.evaluate()

            log = {
            "epoch": f" {epoch+1} /{self.config.epochs}",
            "test_acc": f" {test_acc:.4f}%"
            }

            test_accuracy_log.append(log)

            print(f"Epoch: {epoch+1}/{self.config.epochs} , Test acc: {test_acc:.4f}%")

        with open(test_accuracy_file, 'w') as f:
            json.dump(test_accuracy_log , f)
        

In [13]:
try :
    config = ConfigurationManager()

    model_evaluation_config = config.get_model_evalutaion_config()

    data_transfromation = DataTransformation(model_eval_config)
    data_transfromation.data_augmentation()

    test_dataset = data_transfromation.transformed_dataset()

    model_evaluation = ModelEvaluation(model_evaluation_config, test_dataset= test_dataset)

    print("\n\n -----------Model Evaluation Started----------------------")
    
    model_evaluation.model_test_pipeline()

    print("\n\n -----------Model Evaluation Completed----------------------")



except Exception as e:
    raise e


[2025-08-09 14:24:40,551 : INFO : common  : yaml file config\config.yaml was read succesfully]
[2025-08-09 14:24:40,560 : INFO : common  : yaml file params.yaml was read succesfully]
[2025-08-09 14:24:40,561 : INFO : common  : Created directory at : artifacts]
[2025-08-09 14:24:40,562 : INFO : common  : Created directory at : artifacts/model_evaluation]
[2025-08-09 14:24:40,960 : INFO : 1566778438  : Test Dataset Transformed Successfully]
Test Dataset Transformed Successfully


FileNotFoundError: [Errno 2] No such file or directory: ''